In [1]:
import csv
import os
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

# Cambios en usuarios

## Préstamos

In [2]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'SUCURSAL': str,
    'COD_BARRAS_LIBRO': str,
    'DEWEY': str,
    'DESC_LOCALIZACION_EJEMPLAR': str,
    'FECHA_PUB': str,
    'PAIS_EJEMPLAR': str,
    'OCUPACION': str,
    'NIVEL_EDUCACION': str,
    'AREA_ESTUDIO': str,
    'SUCURSAL_PRESTAMO': str,
    'SUCURSAL_DEVOLUCION': str,
    'SUCURSAL_DEL_LECTOR': str,
    'COD_BARRAS_LECTOR': str,
    'CATEGORÍA': str,
    'FECHA_PRESTAMO': str,
    'FECHA_REAL_DEVOLUCION': str,
    'DATE_PRESTAMO': pd.tslib.Timestamp,
    'DATE_DEVOLUCION': pd.tslib.Timestamp,
}

prestamos_df = pd.read_csv('prestamos_totales.csv', header=0, dtype=dtypes, parse_dates=['DATE_PRESTAMO', 'DATE_DEVOLUCION'])
prestamos_df = prestamos_df.fillna('')

CPU times: user 19.9 s, sys: 600 ms, total: 20.5 s
Wall time: 20.5 s


In [3]:
prestamos_df.shape

(3054360, 20)

In [4]:
prestamos_df.head()

,TITULO,AUTOR,SUCURSAL,COD_BARRAS_LIBRO,DEWEY,DESC_LOCALIZACION_EJEMPLAR,FECHA_PUB,PAIS_EJEMPLAR,OCUPACION,NIVEL_EDUCACION,AREA_ESTUDIO,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION,SUCURSAL_DEL_LECTOR,COD_BARRAS_LECTOR,CATEGORÍA,FECHA_PRESTAMO,FECHA_REAL_DEVOLUCION,DATE_PRESTAMO,DATE_DEVOLUCION
0,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429527,CTA,22/01/2015,04/02/2015,2015-01-22,2015-02-04
1,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88416381,CTA,31/03/2015,07/04/2015,2015-03-31,2015-04-07
2,Quisiera tener ...,"Zoboli, Giovanna.",ARM,2284458,853,Infantil,2010,mx,,,,ARM,ARM,ARM,88381353,CTA,07/11/2015,17/11/2015,2015-11-07,2015-11-17
3,Johannes Gutenberg,"Borràs Perelló, Lluís.",ARM,2288922,925,Infantil,2010,sp,Empleado,Técnico,Adm. de empresas,ARM,ARM,ARM,88007424,CEM,16/01/2015,16/01/2015,2015-01-16,2015-01-16
4,La fantástica leyenda de: la princesa y el dragón,"Alins, Sonia.",ARM,2288896,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429529,CTA,21/01/2015,28/01/2015,2015-01-21,2015-01-28


## Últimos préstamos

In [5]:
%%time

dtypes = {
    'OCUPACION': str,
    'NIVEL_EDUCACION': str,
    'AREA_ESTUDIO': str,
    'SUCURSAL_DEL_LECTOR': str,
    'COD_BARRAS_LECTOR': str,
    'CATEGORÍA': str,
}

ultimos_prestamos_df = pd.read_csv('ultimos_prestamos.csv', header=0, dtype=dtypes, parse_dates=[])
ultimos_prestamos_df = ultimos_prestamos_df.fillna('')

CPU times: user 332 ms, sys: 0 ns, total: 332 ms
Wall time: 331 ms


In [6]:
ultimos_prestamos_df.shape

(156392, 6)

In [7]:
ultimos_prestamos_df.head()

,OCUPACION,NIVEL_EDUCACION,AREA_ESTUDIO,SUCURSAL_DEL_LECTOR,COD_BARRAS_LECTOR,CATEGORÍA
0,,,,ARM,88429527,CTA
1,,,,ARM,88429529,CTA
2,docente,postgrado,psicología,ARM,88209238,CTF
3,estudiante,,,ARM,88439347,CTB
4,,,,ARM,88429493,CTA


## Socios activos 2016

In [8]:
# 1.       CTA: categoría para niños y jóvenes hasta bachillerato
# 2.       CTI: categoría infantil, hasta doce años y solo es para Bogotá (blaa –biblioteca Luis Ángel Arango)
# 3.       CTB: público general (solo permite préstamo de libros)
# 4.       CTC: público en general (permite préstamo de libros y material audiovisual)
# 5.       CTF: categoría familiar, pueden acceder a los beneficios hasta tres personas del grupo familiar que se asocia
# 6.       CE1, CE2, CE2: categorías empresariales (solo para instituciones)
# 7.       CIS: categoría Inclusión Social
# 8.       CTU: categoría Universitaria
# 9.       VIP: Investigadores, préstamo de 30 materiales (libros y material audiovisual)

#      Categoría de lector: CE1 Categoría empresarial                                   218
#      Categoría de lector: CE2 Categoría empresarial 2                                   5
#      Categoría de lector: CEM Categoría Empleado/Pensionado                          1593
#      Categoría de lector: CIN Categoría Interbibliotecario                            128
#      Categoría de lector: CIP Categoría Investigador                                  351
#      Categoría de lector: CIS Categoría Inclusión Preferencial                      15974
#      Categoría de lector: CIV Categoría Investigadores                                 72
#      Categoría de lector: CML Categoría Mediador de Proyectos Culturales                2
#      Categoría de lector: CSU Categoría Sucursal                                       91
#      Categoría de lector: CTA Categoría Básica A                                    10610
#      Categoría de lector: CTB Categoría B                                            1359
#      Categoría de lector: CTC Categoría C                                             224
#      Categoría de lector: CTF Categoría familiar                                     1105
#      Categoría de lector: CTI Categoría Infantil                                     1004
#      Categoría de lector: CTU Categoría General                                     17402
#      Categoría de lector: PTE PROCESOS TÉCNICOS                                        95

In [9]:
%%time

dtypes = {
    'CODBAR': str,
    'Sucursal': str,
    'Tratamiento': str,
    'Barrio': str,
    'Ciudad': str,
    'CATEG': str,
    'Ocupación1': str,
    'Nivel Estudio1': str,
    'Área de Estudio1': str,
    'Fech. Nacim': pd.tslib.Timestamp,
    'Edad': float,
}

perfiles_df = pd.read_csv('socios_activos_2016.csv', header=0, dtype=dtypes, parse_dates=['Fech. Nacim'])
perfiles_df = perfiles_df.fillna('')

CPU times: user 440 ms, sys: 4 ms, total: 444 ms
Wall time: 443 ms


In [10]:
perfiles_df.shape

(50272, 11)

In [11]:
perfiles_df.head()

,CODBAR,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Fech. Nacim,Edad
0,88180782,BLAA,,chapinero,bogota,CIN,universidad,,,,
1,88183608,BLAA,sra.,ayacuyo ii,bogota,CEM,pensionado,,,1957-07-30 00:00:00,59
2,88181732,BLAA,sra.,icata,bogota,CEM,empleado,profesional,filosofía y letras,,
3,88182347,ARM,sr.,montenegro,montenegro,CEM,empleado,profesional,economía,1963-09-20 00:00:00,53
4,88181284,BLAA,sr.,la soledad,bogota,CEM,,,,,


# Cambios

## Préstamos

In [12]:
changes = defaultdict(dict)

In [13]:
usr_columns = ['OCUPACION', 'NIVEL_EDUCACION', 'AREA_ESTUDIO', 'SUCURSAL_DEL_LECTOR', 'CATEGORÍA']
ext_columns = ['COD_BARRAS_LECTOR', 'DATE_PRESTAMO']
aux_columns = usr_columns + ext_columns

In [14]:
prestamos_por_usuario = prestamos_df[aux_columns] \
    .sort_values(ext_columns) \
    .groupby('COD_BARRAS_LECTOR')

In [15]:
%%time

for codbar, group in prestamos_por_usuario:
    for barcode in group['COD_BARRAS_LECTOR']:
        assert(codbar == barcode)
    for c in usr_columns:
        s = group[c]
        for t in zip(s, s[1:]):
            if t[0] and t[1] and t[0] != t[1]:
                changes[c].setdefault(t, 0)
                changes[c][t] += 1

CPU times: user 3min, sys: 540 ms, total: 3min
Wall time: 2min 59s


## Últimos préstamos y socios activos 2016

In [16]:
perfiles_df.set_index('CODBAR', inplace=True, verify_integrity=True)

In [17]:
used_colums_1 = [
    # Ejemplar
    'TITULO', 'AUTOR', 'SUCURSAL', 'COD_BARRAS_LIBRO', 'DEWEY', 'DESC_LOCALIZACION_EJEMPLAR',
    'FECHA_PUB','PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION','NIVEL_EDUCACION','AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO','SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO','FECHA_REAL_DEVOLUCION'
]

In [18]:
used_colums_2 = [
    # Identificación
    'CODBAR',
    # Localización (I)
    'Sucursal',
    # Género
    'Tratamiento',
    # Localización (II)
    'Barrio', 'Ciudad',
    # Educación/Ocupación
    'CATEG', 'Ocupación1', 'Nivel Estudio1', 'Área de Estudio1',
    # Edad
    'Fech. Nacim',
]

In [19]:
mappings = {
    'OCUPACION': 'Ocupación1',
    'NIVEL_EDUCACION': 'Nivel Estudio1',
    'AREA_ESTUDIO': 'Área de Estudio1',
    'SUCURSAL_DEL_LECTOR': 'Sucursal',
    'COD_BARRAS_LECTOR': 'CODBAR',
    'CATEGORÍA': 'CATEG',
}

In [20]:
%%time

for i1, data1 in ultimos_prestamos_df.iterrows():
    codbar = data1['COD_BARRAS_LECTOR']
    if codbar in perfiles_df.index:
        i2 = codbar
        data2 = perfiles_df.ix[i2]
        for c in usr_columns:
            if data1[c] and data2[mappings[c]] and data1[c] != data2[mappings[c]]:
                changes[c].setdefault((data1[c], data2[mappings[c]]), 0)
                changes[c][(data1[c], data2[mappings[c]])] += 1

CPU times: user 29.6 s, sys: 4 ms, total: 29.6 s
Wall time: 29.6 s


In [21]:
changes = dict(changes)

# Tablas

In [22]:
list(changes.keys())

['CATEGORÍA',
 'OCUPACION',
 'NIVEL_EDUCACION',
 'SUCURSAL_DEL_LECTOR',
 'AREA_ESTUDIO']

## Área de estudio

In [23]:
data = [(a, b, c) for ((a, b), c) in changes['AREA_ESTUDIO'].items()]
area_estudio_df = pd.DataFrame(data, columns=['antiguo', 'nuevo', 'frecuencia'])

In [24]:
area_estudio_df.sort_values('frecuencia', ascending=False)

,antiguo,nuevo,frecuencia
351,Sociología,Ciencias sociales,9
338,Literatura,Filosofía y letras,8
218,Filosofía y letras,Derecho,8
153,Idiomas,Literatura,8
548,Ciencias sociales,Historia,8
16,Derecho,Literatura,7
180,Literatura,Derecho,7
126,Sociología,Ciencias políticas,7
164,Literatura,Idiomas,7
34,Ciencias sociales,Pedagogía,6


## Ocupación

In [25]:
data = [(a, b, c) for ((a, b), c) in changes['OCUPACION'].items()]
ocupacion_df = pd.DataFrame(data, columns=['antiguo', 'nuevo', 'frecuencia'])

In [26]:
ocupacion_df.sort_values('frecuencia', ascending=False)

,antiguo,nuevo,frecuencia
96,Estudiante,Docente,229
63,Estudiante,Empleado,226
54,Estudiante,Independiente,197
74,Empleado,Estudiante,106
52,Estudiante,Trabajador y estudiante,100
28,Docente,Estudiante,89
79,Independiente,Estudiante,75
3,Hogar,Estudiante,68
60,Empleado,Docente,57
5,Empleado,Independiente,51


## Categoría

In [27]:
data = [(a, b, c) for ((a, b), c) in changes['CATEGORÍA'].items()]
categoria_df = pd.DataFrame(data, columns=['antiguo', 'nuevo', 'frecuencia'])

In [28]:
categoria_df.sort_values('frecuencia', ascending=False)

,antiguo,nuevo,frecuencia
59,CTB,CTU,2776
60,CTB,CTC,972
87,CTB,CIS,756
2,CTC,CTU,575
72,CTC,CTB,382
93,CTC,CTF,245
16,CTB,CTF,170
5,CTA,CTB,142
56,CTC,CIS,128
94,CIV,CSU,108


## Nivel de educación

In [29]:
data = [(a, b, c) for ((a, b), c) in changes['NIVEL_EDUCACION'].items()]
nivel_educacion_df = pd.DataFrame(data, columns=['antiguo', 'nuevo', 'frecuencia'])

In [30]:
nivel_educacion_df.sort_values('frecuencia', ascending=False)

,antiguo,nuevo,frecuencia
27,Universitario 1 nivel,Universitario 2 nivel,1237
11,Universitario 2 nivel,Profesional,396
53,Básica primaria,Secundaria,314
56,Secundaria,Universitario 1 nivel,207
79,Profesional,Maestría,206
29,Universitario 1 nivel,Profesional,134
58,Universitario 2 nivel,Universitario 1 nivel,130
37,Universitario 2 nivel,Maestría,121
5,Profesional,Universitario 1 nivel,89
62,Profesional,Postgrado,81


## Sucursal del lector

In [31]:
data = [(a, b, c) for ((a, b), c) in changes['SUCURSAL_DEL_LECTOR'].items()]
sucursal_lector_df = pd.DataFrame(data, columns=['antiguo', 'nuevo', 'frecuencia'])

In [32]:
sucursal_lector_df.sort_values('frecuencia', ascending=False)

,antiguo,nuevo,frecuencia
219,BLAA,CASAGO,412
40,CASAGO,BLAA,190
167,BLAA,CAL,51
159,BLAA,TUN,47
97,BLAA,MED,31
200,TUN,BLAA,26
18,BUC,BLAA,24
137,BLAA,IBA,23
149,BLAA,BUC,23
208,IBA,BLAA,22
